# Coding 11: Imitation Learning

This week we will train a deep network that learns to race in SuperTuxKart using only the camera and speedometer.

The network will take in an image and directly regress to low-level controls (steering, acceleration, brake, drift).

<img src="https://a.fsdn.com/con/app/proj/supertuxkart/screenshots/500px-Hac.jpg/max/max/1" width=512px/>

This coding exercise consists of three main parts -

* collecting data
* training your model

## The Environment

The following code wraps the `pystk` library and provides a basic interface for playing the game.  
No need to modify any of the code - just read over the `rollout` method to see how your agent is used.

In [ ]:
import pystk
import numpy as np
import random

from tqdm.notebook import tqdm


class PyTux(object):
    INITED = False

    def __init__(self, track, screen_width=128, screen_height=96):
        self.race = None
        self.config = pystk.GraphicsConfig.ld()
        self.config.screen_width = screen_width
        self.config.screen_height = screen_height
        self.track = track

        if not PyTux.INITED:
            pystk.init(self.config)
            PyTux.INITED = True

    @staticmethod
    def _magical_auto_pilot(player, track, distance=10):
        """
        This function return a magical steering, acceleration and drift values.
        This is used in the auto-pilot and meant to be hard to read ;)
        Feel free to get inspired by this if you can decipher it
        (it's probably not worth your time though)
        """
        __ = PyTux._point_on_track(player.kart.distance_down_track+distance, track)
        __ = __ - np.array(player.kart.location)
        _ = np.array(player.kart.front) - np.array(player.kart.location)
        _ = _ / max(np.linalg.norm(_), 1e-10)
        _ = np.cross([0,1,0], _)
        return lambda ___: (_.dot(__), int(___<15), abs(__.dot(_))>1)

    @staticmethod
    def _point_on_track(distance, track):
        node_idx = np.searchsorted(track.path_distance[..., 1], distance % track.path_distance[-1, 1]) % len(track.path_nodes)
        d = track.path_distance[node_idx]
        x = track.path_nodes[node_idx]
        t = (distance - d[0]) / (d[1] - d[0])
        return x[1] * t + x[0] * (1 - t)

    def clean(self):
        if self.race is not None:
            self.race.stop()
            del self.race

    def __enter__(self):
        self.clean()

        config = pystk.RaceConfig(num_kart=1, laps=1, track=self.track, step_size=0.1)
        config.players[0].controller = pystk.PlayerConfig.Controller.PLAYER_CONTROL

        self.race = pystk.Race(config)
        self.race.start()
        self.race.step()

        return self

    def __exit__(self, type, value, traceback):
        self.clean()
        pystk.clean()
        PyTux.INITED = False

        return True

    def rollout(self, agent, max_frames=1000):
        """
        agent: an object that implements the act method
        max_frames: maximum number of frames to play for

        returns: tuple of (number steps, overall distance, did the agent finish)
        """
        state = pystk.WorldState()
        track = pystk.Track()

        for t in tqdm(range(max_frames)):
            state.update()
            track.update()

            player = state.players[0]

            # Terminate if the kart finishes a lap.
            if np.isclose(player.kart.overall_distance / track.length, 1.0, atol=2e-3):
                return t, player.kart.overall_distance, True

            image = np.array(self.race.render_data[0].image)
            auto_pilot = self._magical_auto_pilot(player, track)
            speed = np.linalg.norm(player.kart.velocity)

            self.race.step(agent.act(image, auto_pilot, speed))
        return t, 100 * player.kart.overall_distance / track.length, False

## The Agent

Here we'll implement our first agent - the autopilot.  
All we need to do for the autopilot is implement the `act` method.  

This method uses the `target` waypoint to compute an action, which consists of steering, acceleration, brake, and drift.

**QUESTION:** what coordinates is `target` in?

Aside: the autopilot gets *ground truth information* and is allowed to know the `target` every time step.  
Isn't this cheating?

In [ ]:
class AgentWrapper(object):
    """
    Wraps any agent to collect extra information, used for
    - collecting data
    - visualizing runs
    """
    ACTIONS = ['steer', 'acceleration', 'brake', 'drift']

    def __init__(self, agent):
        self.agent = agent

        self.images = list()
        self.actions = list()

    def act(self, image, target, speed):
        action = self.agent.act(image, target, speed)

        self.images.append(image.copy())
        self.actions.append([getattr(action, x) for x in self.ACTIONS])

        return action

    def show(self):
        """
        Call on the last line of a cell to visualize the most recent run.
        """
        from moviepy.editor import ImageSequenceClip
        from IPython.display import display

        display(ImageSequenceClip(self.images, fps=15).ipython_display(width=512, autoplay=True, loop=True))
        
        
class Autopilot(object):
    last_drift = False
    def act(self, image, auto_pilot, speed):
        action = pystk.Action()
        # Let's use a magical auto-pilot function to get the correct actions
        steer, acceleration, drift = auto_pilot(speed)
        # Let's make sure the auto-pilot's actions fall within our limits
        action.steer = np.clip(steer, -1, 1)
        action.acceleration = np.clip(acceleration,0,1)
        # Cheap trick, never drift twice in a row (you'll slide out of control)
        action.drift = int(drift) and not self.last_drift
        self.last_drift = action.drift
        action.brake = 0
        return action


TRACK = 'lighthouse'
TRACK_TIME = 700

# Test out your controller!
agent = AgentWrapper(Autopilot())

with PyTux(TRACK) as env:
    print('Time: %d Distance: %d Success: %d' % env.rollout(agent, TRACK_TIME))

agent.show()

## Data Collection

Now that we have a decent autopilot, let's collect some data.  

For every frame we have the following data
* image
* action

All we have to do is let the autopilot drive a couple times,  
and we'll save this information from every frame.

No need to modify anything here besides `N_EPISODES`.

In [ ]:
from pathlib import Path
from PIL import Image


DATASET_PATH = Path('drive_data')
N_EPISODES = 1

# Remove old data.
!rm -rf $DATASET_PATH

DATASET_PATH.mkdir()

for episode in tqdm(range(N_EPISODES)):
    # Reset the agent.
    agent = AgentWrapper(Autopilot())

    # Run one rollout.
    with PyTux(TRACK) as env:
        env.rollout(agent, TRACK_TIME)

    # Save the data.
    episode_dir = Path(DATASET_PATH) / ('%03d' % episode)
    episode_dir.mkdir()

    image_dir = episode_dir / 'images'
    image_dir.mkdir()

    for i, image in enumerate(agent.images):
        Image.fromarray(image).save(image_dir / ('%05d.png' % i))

    (episode_dir / 'actions.txt').write_text('\n'.join(map(str, agent.actions)))

## Dataset

Now that the data is collected, we'll load it using our standard SuperTuxDataset (from HW1-3).

No need to change anything here, just take a look at the `visualize_sample` code as always.

In [ ]:
%matplotlib inline

import pathlib
import torch
import torchvision
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw


class SuperTuxDataset(torch.utils.data.Dataset):
    def __init__(self, episode_dir, transform=torchvision.transforms.ToTensor()):
        episode_dir = pathlib.Path(episode_dir)

        self.transform = transform
        self.image_paths = list(sorted((episode_dir / 'images').glob('*.png')))
        self.actions = list()

        for line in (episode_dir / 'actions.txt').read_text().split('\n'):
            self.actions.append(eval(line))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        image = self.transform(image)

        action = torch.FloatTensor(self.actions[idx])

        return image, action


def load_data(dataset_path, batch_size=128, transform=torchvision.transforms.ToTensor()):
    dataset = list()

    for episode_dir in pathlib.Path(dataset_path).glob('*'):
        data = SuperTuxDataset(episode_dir, transform)
        dataset.append(data)

    dataset = torch.utils.data.ConcatDataset(dataset)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)


@torch.no_grad()
def visualize_sample(image, action):
    image = (255 * image).byte().numpy().transpose(1, 2, 0)
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)

    for i, a in enumerate(action):
        draw.text((5, 5 + 15 * i), str(a.item()), fill=(255, 0, 0))

    return np.array(image)


n_samples = 4
fig, axes = plt.subplots(1, n_samples)
fig.set_size_inches(20, 10)

data = SuperTuxDataset(DATASET_PATH / '000')

for i in range(n_samples):
    idx = np.random.randint(len(data))
    axes[i].imshow(visualize_sample(*data[idx]))

plt.show()

## Tensorboard

In [ ]:
import torch.utils.tensorboard as tb

log_dir = 'log'

%load_ext tensorboard
%tensorboard --logdir {log_dir} --reload_interval 1

## Model + Training

Now we can finally define our model.  
The model takes in an image and outputs an action.

In [ ]:
import time
import torch

from torch.utils.tensorboard import SummaryWriter


class CNNClassifier(torch.nn.Module):
    def __init__(self, c_in, c_out):
        super().__init__()
        # TODO: Implement this -- Your network does not need to be as big as other networks
        # we've looked at in this class

    def forward(self, x):
        pass

    def act(self, image, _, speed):
        """
        Implement this.

        Remember that image comes in as a np.uint8 array, with shape (h, w, 3).
        The values range from [0-255]
        """
        action = pystk.Action()
        # Use your network to set the following:
        #action.steer, float between -1 and 1 -- the direction to steer
        #action.acceleration, float between 0 and 1 -- the amount to accelerate
        #action.drift, bool -- whether or not to drift, which can help with tight turns
        #action.brake, bool -- whether or not to brake
        return action


def train(model, device, lr=0.001, epochs=1000):
    logger = tb.SummaryWriter(log_dir + '/{}'.format(time.strftime('%H-%M-%S')), flush_secs=1)

    optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
    global_step = 0

    # Is this the right loss?
    loss_func = torch.nn.MSELoss()
    
    model.to(device)
    
    for epoch in tqdm(range(epochs)):
        model.train()
        
        for image, action in data_train:
            image, action = image.to(device), action.to(device)
            pred = model(image)
            
            # NOTE: The following loss function ignores the drift and brake outputs
            # of your network. This can get you a pretty good controller and is a bit
            # easier to train.
            loss = loss_func(pred[..., :2], action[..., :2])

            optim.zero_grad()
            loss.backward()
            optim.step()

            logger.add_scalar('loss/train', loss.item(), global_step)
            global_step += 1

            # Add image visualization.
            # visualize_sample(image, action)


# Train your model.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_train = load_data('drive_data', batch_size=32)

model = CNNClassifier(3, 4)
model.to(device)

train(model, device)

## Fully Autonomous Driving

Now that all that has been taken care of, it's time to finally test out your trained model.

In [ ]:
model.eval()
model = model.cpu()
model.GAIN=1.2
model.ACCEL_GAIN=100

# Test out your model!
with PyTux(TRACK) as env:
    agent = AgentWrapper(model)
    print('Time: %d Distance Completed Percentage: %f Success: %d' % env.rollout(agent, TRACK_TIME))

agent.show()